In [7]:
    # Make a prediction with a decision tree
    def predict(node, row):
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict):    #if value corresponding to the key 'left' is a dictionary we proceed recursively
                return predict(node['left'], row) #else we make a prediction
            else:
                return node['left']
        else:
            if isinstance(node['right'], dict):
                return predict(node['right'], row)
            else:
                return node['right']
    # Split a dataset based on an attribute and an attribute value
    def test_split(index, value, dataset):
        left, right = list(), list()
        for row in dataset:
            if row[index] < value:
                left.append(row)
            else:
                right.append(row)
        return left, right

    # Calculate the Gini index for a split dataset
    def gini_index(groups, classes):
        # count all samples at split point
        n_instances = float(sum([len(group) for group in groups]))
        # sum weighted Gini index for each group
        gini = 0.0
        for group in groups:
            size = float(len(group))
            # avoid divide by zero
            if size == 0:
                continue
            score = 0.0
            for class_val in classes:#gini split=(|d1|/|d|)*gini(d1)+(|d2|/|d|)*gini(d2)
                p = [row[-1] for row in group].count(class_val) / size
                score += p * p
            # weight the group score by its relative size
            gini += (1.0 - score) * (size / n_instances)
        return gini

    # Select the best split point for a dataset
    def get_split(dataset,depth):
        print("level=",depth)
        l=[row[-1] for row in dataset]
        print("count of 0=",l.count(0))
        print("count of 1=",l.count(1))
        print("count of 2=",l.count(2))
        class_values = list(set(row[-1] for row in dataset))
        b_index, b_value, b_score, b_groups = 999, 999, 999, None
        #we try to split about every value of every feature in order to choose the best possible split point
        for index in range(len(dataset[0])-1): 
            for row in dataset:
                groups = test_split(index, row[index], dataset)
                gini = gini_index(groups, class_values)
                if gini < b_score:
                    b_index, b_value, b_score, b_groups = index, row[index], gini, groups
        print("splitting on feature %d about %.3f with gini index=%.3f" %(b_index,b_value,b_score))
        print()
        return {'index':b_index, 'value':b_value, 'groups':b_groups} #using dictionary to represent a node in the decision tree

    # Create a terminal node value
    def to_terminal(group,depth):
        print("level:",depth)
        l=[row[-1] for row in group]
        print("count of 0=",l.count(0))
        print("count of 1=",l.count(1))
        print("count of 2=",l.count(2))
        print("Terminal node")
        print()
        outcomes = [row[-1] for row in group]
        return max(set(outcomes), key=outcomes.count) #This function returns the most common output value in a list of rows.

    # Create child splits for a node or make terminal
    def split(node, max_depth, min_size, depth):
        left, right = node['groups']
        del(node['groups'])
        # we check if either left or right group of rows is empty and if so we create a terminal node
        if not left or not right:
            node['left'] = node['right'] = to_terminal(left + right,depth)#We will refer to the child nodes as left and
                                                                    #right in the dictionary representation of a given node.
            return
        # check for max depth
        if depth >= max_depth:
            node['left'], node['right'] = to_terminal(left,depth), to_terminal(right,depth)
            return
        # process left child
        if len(left) <= min_size:
            node['left'] = to_terminal(left,depth)
        else:
            node['left'] = get_split(left,depth)
            split(node['left'], max_depth, min_size, depth+1)
        # process right child
        if len(right) <= min_size:
            node['right'] = to_terminal(right,depth)
        else:
            node['right'] = get_split(right,depth)
            split(node['right'], max_depth, min_size, depth+1)

    # Build a decision tree
    def build_tree(train, max_depth, min_size):
        root = get_split(train,0)
        split(root, max_depth, min_size, 1)
        return root

    # Print a decision tree
    def print_tree(node, depth=0):
        if isinstance(node, dict):#The isinstance() function checks if the object (first argument) is an instance or
            #subclass of classinfo class (second argument).So we are checking whether the node is a dictionary(non-terminal node)
            #or a terminal node(an output)
            print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))#Here the number of spaces indicate the level
            print_tree(node['left'], depth+1)                                        #we are operating at.
            print_tree(node['right'], depth+1)
        else:
            print('%s[%s]' % ((depth*' ', node))) #for terminal nodes we print the corresponding outputs.
    from sklearn import datasets
    iris=datasets.load_iris()
    from sklearn.model_selection import train_test_split
    train,test,y_train,y_test=train_test_split(iris.data,iris.target,random_state=0)
    train=train.tolist() #converting the numpy arrays to list in order to append input and output arrays to form a dataset
    test=test.tolist()
    y_train=y_train.tolist()
    y_test=y_test.tolist()
    for i in range(len(train)):
        train[i].append(y_train[i])
    for i in range(len(test)):
        test[i].append(y_test[i])  
    tree = build_tree(train, 4, 1) # building the tree using training dataset
    print_tree(tree)
    predictions=list()
    for row in test:#using test dataset to make predictions
        prediction = predict(tree, row)
        predictions.append(prediction)
    from sklearn.metrics import confusion_matrix
    print(confusion_matrix(y_test,predictions))


level= 0
count of 0= 37
count of 1= 34
count of 2= 41
splitting on feature 2 about 3.000 with gini index=0.332

level= 1
count of 0= 37
count of 1= 0
count of 2= 0
splitting on feature 0 about 4.700 with gini index=0.000

level= 2
count of 0= 7
count of 1= 0
count of 2= 0
splitting on feature 0 about 4.300 with gini index=0.000

level: 3
count of 0= 7
count of 1= 0
count of 2= 0
Terminal node

level= 2
count of 0= 30
count of 1= 0
count of 2= 0
splitting on feature 0 about 4.700 with gini index=0.000

level: 3
count of 0= 30
count of 1= 0
count of 2= 0
Terminal node

level= 1
count of 0= 0
count of 1= 34
count of 2= 41
splitting on feature 2 about 5.000 with gini index=0.099

level= 2
count of 0= 0
count of 1= 33
count of 2= 3
splitting on feature 3 about 1.700 with gini index=0.042

level= 3
count of 0= 0
count of 1= 32
count of 2= 0
splitting on feature 0 about 5.900 with gini index=0.000

level: 4
count of 0= 0
count of 1= 19
count of 2= 0
Terminal node

level: 4
count of 0= 0
count

In [1]:
# print("splitting on feature %d about %.3f with gini index=%.3f" %(1,0.36474,2.3848))

splitting on feature 1 about 0.365 with gini index=2.385


In [5]:
# def func(x):
#     del(a['nick'])
#     a['fav']="JT"
# a={'name':'pri','age':19,"nick":'pi'}
# func(a)
# print(a)

{'name': 'pri', 'age': 19, 'fav': 'JT'}
